# Project 2
# Mini Team 1

In [2]:
import pandas as pd
import numpy as np
import hashlib

# TASK 1: Load Dataset

In [3]:
# TASK 1: Load Dataset

df = pd.read_csv('/content/diabetic_data (1).csv')
df.head()


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


# TASK 2: Handle Sensitive Data (Anonymization)

In [4]:
# TASK 2: Anonymize Sensitive Healthcare Data

def hash_id(x):
    return hashlib.sha256(str(x).encode()).hexdigest()

df['anonymous_patient_id'] = df['patient_nbr'].apply(hash_id)

df.drop(['patient_nbr', 'encounter_id'], axis=1, inplace=True)


# TASK 3: Merge Multiple Data Sources

In [10]:
# TASK 3: Merge Mapping Tables

ids = pd.read_csv('/content/IDS_mapping.csv')

# Convert the 'admission_type_id' column in df to string type to match 'ids' DataFrame
df['admission_type_id'] = df['admission_type_id'].astype(str)

df = df.merge(ids, on='admission_type_id', how='left')
df.rename(columns={'description': 'admission_type'}, inplace=True)

# TASK 4: Handle Missing Values

In [17]:
# TASK 4: Handle Missing Values

df.replace('?', np.nan, inplace=True)

missing_ratio = df.isna().mean()
df.drop(missing_ratio[missing_ratio > 0.5].index, axis=1, inplace=True)

cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    # Convert column to string type to ensure consistent data for mode calculation
    temp_series = df[col].astype(str)

    # Calculate the mode. Handle potential empty mode result explicitly.
    modes = temp_series.mode()
    if not modes.empty:
        # Access the first mode value from the Series
        mode_val = modes.iloc[0]
    else:
        # Fallback for columns where mode cannot be determined (e.g., all values are NaNs)
        # Using a generic placeholder 'Unknown' for such cases.
        mode_val = 'Unknown'

    df[col] = df[col].fillna(mode_val)

num_cols = df.select_dtypes(include='number').columns
for col in num_cols:
    # Assign the result back to the column instead of using inplace=True
    df[col] = df[col].fillna(df[col].median())

# TASK 5: Handle Outliers

In [18]:
# TASK 5: Handle Outliers using IQR

def iqr_filter(data, col):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[col] >= lower) & (data[col] <= upper)]

for col in ['num_lab_procedures', 'num_medications', 'time_in_hospital']:
    df = iqr_filter(df, col)


# TASK 6: Data Quality Assessment

In [19]:
# TASK 6: Data Quality Assessment Report

data_quality = pd.DataFrame({
    'Column': df.columns,
    'Data Type': df.dtypes.astype(str),
    'Missing %': df.isna().mean() * 100,
    'Unique Values': df.nunique()
})

data_quality


,Column,Data Type,Missing %,Unique Values
race,race,object,0.0,5
gender,gender,object,0.0,3
age,age,object,0.0,10
admission_type_id,admission_type_id,object,0.0,8
discharge_disposition_id,discharge_disposition_id,int64,0.0,26
admission_source_id,admission_source_id,int64,0.0,17
time_in_hospital,time_in_hospital,int64,0.0,12
payer_code,payer_code,object,0.0,16
medical_specialty,medical_specialty,object,0.0,72
num_lab_procedures,num_lab_procedures,int64,0.0,96


# TASK 7: Export Clean Dataset

In [20]:
# TASK 7: Export Final Dataset

df.to_csv('clean_anonymized_readmission_data.csv', index=False)
